<a href="https://colab.research.google.com/github/KartikLal/Text-Mining-of-Stack-Overflow/blob/master/StackOverFlow_TextMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [0]:
import pandas as pd
import tensorflow as tf
import re
import nltk
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.cluster import KMeans
from nltk.cluster import KMeansClusterer
import nltk

In [0]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Importing data from drive

In [0]:
from google.colab import drive
drive.mount('/gdrive')

KeyboardInterrupt: ignored

In [0]:
train_df = pd.read_csv('/content/Train_sample.csv')

In [0]:
display(train_df.head())

,Id,Title,Body,Tags
0,1319228,Assign unique ID to a DefaultMutableTreeNode i...,<p>I have a Jtree that I can add new Nodes to....,java swing jtree treenode
1,473636,"Does anyone know if ""SATA II"" drives are compa...",<p>I want to buy some cheaper HDs for our serv...,sata server-configuration dell-poweredge sas
2,5423128,iOS 5 segue initialize more than one property,<p>i did segue with one property it works but ...,objective-c xcode ios5
3,5112680,Can content be recovered from Chrome's cache?,<p>I've been working on a draft post in Blogge...,browser-cache temporary-files
4,735565,How can I programmatically detect side effects...,<p>I've got an idea for caching that I'm begin...,c# memoization side-effects


In [0]:
train_X = train_df['Title'].tolist()

train_labels = train_df['Tags']

# DATA PREPROCESSING

## Removing html tags

In [0]:
#remove html tags
def strip_html_tag(sample):
    cleanr = re.compile(r"\<[^\>]*\>|\\")
    return re.sub(cleanr, '', sample)

In [0]:
train_X = list(map(strip_html_tag, train_X))

## Converting all text to lowercase

> Indented block

> Indented block





In [0]:
#Converting string to lower case
def string_lower(sample):
    sample.replace('.',' . ')
    return sample.lower()

In [0]:
train_X = list(map(string_lower, train_X))
# test_X = list(map(string_lower, test_X))

## Tokenize all texts

In [0]:
# Tokenize
def tokenize_text(sample):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(sample)

In [0]:
train_X = list(map(tokenize_text,train_X))
# test_X = list(map(tokenize_text,test_X))

## Remove stop words

In [0]:
# Remove stop words
def stopword_remover(sample):
    stopwords_list = set(stopwords.words('english'))
    sample_wostopword = []
    for i in sample:
        if i not in stopwords_list:
            sample_wostopword.append(i) 
    return sample_wostopword

In [0]:
train_X = list(map(stopword_remover,train_X))
# test_X = list(map(stopword_remover,test_X))

## Remove punctuations

In [0]:
# Remove punctuations
def punctuation_remover(sample):
    sample_wostopword_wopunc = []
    for i in sample:
        if i not in string.punctuation:
            sample_wostopword_wopunc.append(i)
    return sample_wostopword_wopunc

In [0]:
train_X = list(map(punctuation_remover,train_X))
# test_X = list(map(punctuation_remover,test_X))

## Adding parts of speech to each word

In [0]:
# Parts of speech
def text_pos_tag(sample):
    return nltk.pos_tag(sample)

In [0]:
train_X = list(map(text_pos_tag,train_X))
# test_X = list(map(text_pos_tag,test_X))

## Lemmatize tokens

In [0]:
def lemmatize(token):
    word, tag = token
    
    lemmatizer = WordNetLemmatizer()
    
    if word not in list(".,;'\"-"):
        return lemmatizer.lemmatize(word, get_wordnet_pos(tag))
    
    return word

In [0]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
# Lemmatize all reviews
def lemmatized_text(sample):
    return list(map(lemmatize,sample))

In [0]:
train_X = list(map(lemmatized_text,train_X))
# test_X = list(map(lemmatized_text,test_X))

## Cleaned text

In [0]:
# Combine token to form sentences as countvactorizer need sentence as input
train_X = list(map(lambda x: " ".join(x), train_X))
# test_X = list(map(lambda x: " ".join(x), test_X))

# Convert text to document term matrix

## tfidf

In [0]:
# keras text to matrix
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_X)

In [0]:
x_train = tokenizer.texts_to_matrix(train_X, mode='tfidf')
# x_test = tokenizer.texts_to_matrix(test_X, mode='tfidf')

In [0]:
print(x_train.shape)
print(tokenizer.word_index)
# print(x_test.shape)

(50000, 20633)
{'use': 1, 'file': 2, 'c': 3, 'get': 4, 'jquery': 5, 'server': 6, 'error': 7, 'php': 8, 'data': 9, 'android': 10, 'work': 11, 'value': 12, 'net': 13, '2': 14, 'java': 15, 'create': 16, 'function': 17, 'window': 18, 'change': 19, 'image': 20, 'page': 21, 'add': 22, 'set': 23, 'string': 24, 'application': 25, 'object': 26, 'sql': 27, 'javascript': 28, 'table': 29, 'find': 30, 'class': 31, '1': 32, 'list': 33, 'way': 34, 'code': 35, 'mysql': 36, '3': 37, 'text': 38, 'user': 39, 'multiple': 40, 'array': 41, 'app': 42, 'one': 43, 'asp': 44, 'run': 45, 'form': 46, 'access': 47, 'html': 48, 'method': 49, 'make': 50, 'query': 51, 'web': 52, 'python': 53, 'database': 54, 'call': 55, 'problem': 56, 'variable': 57, 'load': 58, 'view': 59, 'type': 60, 'name': 61, 'time': 62, 'custom': 63, 'number': 64, '0': 65, 'show': 66, 'select': 67, 'return': 68, 'display': 69, 'update': 70, 'without': 71, 'element': 72, 'x': 73, 'button': 74, 'issue': 75, 'two': 76, 'rail': 77, 'xml': 78, '4': 

In [0]:
x_train.head(5)

AttributeError: ignored

## Count Vectorizer

In [0]:
vectorizer = CountVectorizer(max_features=10000)
cv_train = vectorizer.fit_transform(train_X).toarray()
# cv_test = vectorizer.fit_transform(test_X).toarray()

In [0]:
cv_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Kmeans

## Training

In [0]:
# Execution time around 350 seconds - for 50000 texts
kmeans = KMeans(n_clusters=3).fit(cv_train)

## Labels and centroid of clusters

In [0]:
labels=kmeans.labels_
centroids=kmeans.cluster_centers_

In [0]:
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(cv_train))

Cluster id labels for inputted data
[0 0 0 ... 0 0 0]
Centroids data
[[ 5.27615389e-04  2.11046156e-04  4.22092312e-05 ...  2.11046156e-04
   4.22092312e-05  1.26627693e-04]
 [ 2.92734587e-18  7.53579503e-04  1.12485975e-18 ... -2.54787511e-18
   1.12485975e-18 -3.11708125e-19]
 [ 7.75193798e-04  1.73472348e-18  1.11130723e-18 ... -3.27971157e-18
   1.11130723e-18 -5.42101086e-20]]
Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):
-276000.58706323296


## Analysis of cluster id

In [0]:
df_tags = pd.DataFrame(data = {"que":train_df['Title'],"tags": train_df['Tags'], "cluster_id": labels})

In [0]:
df_tags['cluster_id'].value_counts()

1    43612
0     4248
2     1265
3      875
Name: cluster_id, dtype: int64

In [0]:
pd.set_option('display.max_colwidth', 10000)
df_tags[df_tags['cluster_id'] == 3].sample(n=10)

,que,tags,cluster_id
6513,Retransforming classes on remote JVM using Java agent,javaagents,3
43461,Use of GPS Classes Events in other classes,c# windows-phone-7,3
29772,Initializing struct within a class,c++,3
34119,Add Rect from another class in c#,c# wpf,3
21641,Class Name and File Name,python class filenames,3
19387,View holder Class in Android,android,3
26918,set property of base class in derived class with spring annotations,java spring annotations,3
40099,Pesky new lines and whitespace in XML reader class,iphone objective-c iphone-sdk-4.0 nsdictionary nsxmlparser,3
1177,PHP Constants in Classes,php oop class constants,3
28480,jQuery - How to search within a class for a partial match and remove that class only?,jquery class table-row,3


In [0]:
NUM_CLUSTERS=8
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(cv_train, assign_clusters=True)
print (assigned_clusters)

/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in true_divide
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))
/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


KeyboardInterrupt: ignored

In [0]:
abc=dict()
count=0
for i in tokenizer.word_index:
  abc[i]=cv_train[count].sum()
  count+=1

In [0]:
abc

In [0]:
import collections
clustering = collections.defaultdict(list)

In [0]:
for idx, label in enumerate(kmeans.labels_):
        clustering[label].append(idx)

In [0]:
dict(clustering)

In [0]:
centroids= kmeans.cluster_centers_
print(centroids)

In [0]:
plt.scatter( c= kmeans.labels_.astype(float), s=50, alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
plt.show()